In [1]:
import ROOT as r
import numpy as np

Welcome to JupyROOT 6.22/06


Function below are defined with the help of `momentumFunc.ipynb` on GitHub.

In [2]:
def thetaPhi():
    '''returns random theta and phi values'''
    
    theta = np.random.uniform(0,2*np.pi)
    phi = np.random.uniform(0,np.pi)
    return theta, phi

def momenta(mag, theta, phi):
    '''given an input magnitude, theta and phi the cartesian momenta are returned'''
    
    Px = mag * np.sin(theta) * np.cos(phi) 
    Py = mag * np.sin(theta) * np.sin(phi)
    Pz = mag * np.cos(theta)
    
    return Px, Py, Pz

def fVector(g, theta, phi, mag, mass):
    '''given a tlorentzvector g the cartesian components are set with the spherical components and the mass'''
    
    Px, Py, Pz = momenta(mag, theta, phi)
    g.SetPxPyPzE(Px, Py, Pz, mass)

Following function is also taken from the same file, but just simplified to generate only electron components. The `lorV` is a TLorentzVector, later defined. It can probably be written more efficiently.

In [3]:
def components(num):
    ''' given a number of events wanted, returns arrays of the components of the electron
        after decaying from stationary W boson.
    '''
    
    #initialising arrays for components
    ePx = np.zeros(num)
    ePy = np.zeros(num)
    ePz = np.zeros(num)    
    
    # looping over number of events and filling component arrays
    for i in range(num):
        #making e components
        thetae, phie = thetaPhi()
        fVector(lorV, thetae, phie, 40.15, 40.15)
        
        ePx[i] = lorV.Px()
        ePy[i] = lorV.Py()
        ePz[i] = lorV.Pz()
        
    return ePx, ePy, ePz

The only useful TLorentzVector is `lorV` as we use it to generate given number of electron 3Vectors. The rest is just there, so we can be consistent with more complicated case where we will need all of the TLorentzVectors.

In [4]:
w4v = r.TLorentzVector()
el4v = r.TLorentzVector()
lorV = r.TLorentzVector()

Generating 1000 different cases.

In [5]:
ePx,ePy,ePz = components(10000)

Now we set up tree and branches. We store 4-element arrays into branches. Each branch is one particle. For each event, there will be 2 branches (W boson and Electron), and each branch will have 4 entries (Px,Py,Pz,E). 

In [6]:
tree = r.TTree("tree", "initial random e and n components")

wArr = np.zeros(4)
eArr = np.zeros(4)

tree.Branch("wBoson",wArr,"wArray[4]/D")
tree.Branch("electron",eArr,"elArray[4]/D")

Following code will assign values to our arrays and then fills them into branches.

In [7]:
wArr[3] = 80.3
eArr[3] = wArr[3]/2

for i in range(len(ePx)):
    
    eArr[0] = ePx[i]
    eArr[1] = ePy[i]
    eArr[2] = ePz[i]
   
    tree.Fill()

In [8]:
# Tree in form:
# Row     Instance   W boson    electron
# e.g.:
# 0       0 (=Px)        ...      ...
# 0       1 (=Py)        ...      ...
# 0       2 (=Pz)        ...      ...
# 0       3 (=E)         ...      ...
#tree.Scan() 

This code creates a file and writes a tree into it.

In [9]:
file = r.TFile("generatedComponents.root","recreate")
tree.Write()
file.Close